In [ ]:
#r "nuget:Microsoft.ML"

Installed Packages Microsoft.ML, 1.6.0

In [ ]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
public class WineQualityData
{
    [LoadColumn(0, 10)]
    public float Features { get; set; }
    
    [LoadColumn(11)]
    public float Quality { get; set; }
}

In [ ]:
var dataPath = "../datasets/winequality-red.csv";

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
var data = mlContext.Data.LoadFromTextFile<WineQualityData>(dataPath, hasHeader: true, separatorChar: ';');

In [ ]:
var partitions = mlContext.Data.TrainTestSplit(data, testFraction: 0.3);

var trainingDataView = partitions.TrainSet;
var testDataView = partitions.TestSet;

In [ ]:
var pipeline = mlContext.Transforms.Conversion.MapValueToKey(inputColumnName: "Quality", outputColumnName: "Label")
              .Append(mlContext.Transforms.Concatenate("Features", nameof(WineQualityData.Features)))
              .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"))
              .AppendCacheCheckpoint(mlContext);

In [ ]:
var trainer = mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy("Label", "Features");
var trainingPipeline = pipeline.Append(trainer).Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

In [ ]:
var trainedModel = trainingPipeline.Fit(partitions.TrainSet);

In [ ]:
var predictions = trainedModel.Transform(partitions.TestSet);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions);

In [ ]:
Console.WriteLine($"Micro Accuracy: {metrics.MicroAccuracy:P2}");
Console.WriteLine($"Macro Accuracy: {metrics.MacroAccuracy:P2}");
Console.WriteLine($"Log Loss: {metrics.LogLoss:P2}");
Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());

Micro Accuracy: 59,27%
Macro Accuracy: 26,55%
Log Loss: 97,20%

Confusion table
          ||================================================
PREDICTED ||     0 |     1 |     2 |     3 |     4 |     5 | Recall
TRUTH     ||================================================
        0 ||   165 |    33 |     1 |     0 |     0 |     0 | 0,8291
        1 ||    87 |   114 |     5 |     0 |     0 |     0 | 0,5534
        2 ||     3 |    42 |    12 |     0 |     0 |     0 | 0,2105
        3 ||    13 |     5 |     0 |     0 |     0 |     0 | 0,0000
        4 ||     0 |     3 |     3 |     0 |     0 |     0 | 0,0000
        5 ||     4 |     1 |     0 |     0 |     0 |     0 | 0,0000
          ||================================================
Precision ||0,6066 |0,5758 |0,5714 |0,0000 |0,0000 |0,0000 |

